In [2]:
import cv2
import numpy as np

In [24]:
video = cv2.VideoCapture(filename="videos/road_line2.mp4")
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer = cv2.VideoWriter(filename="videos/linedetection.mp4",
                         fourcc=cv2.VideoWriter_fourcc(*"DIVX"),
                         fps=25,
                         frameSize=(width, height))


def region_of_interest(image, vertices):
    mask = np.zeros_like(image)
    match_mask_color = 255
    
    cv2.fillPoly(img=mask, pts=vertices, color=match_mask_color)
    masked_image = cv2.bitwise_and(src1=image, src2=image, mask=mask)
    return masked_image


def drawLines(image, lines):
    image = np.copy(image)
    blank_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img=image,
                     pt1=(x1, y1),
                     pt2=(x2, y2),
                     color=(0, 255, 0),
                     thickness=10)
    image = cv2.addWeighted(src1=image, alpha=0.8, src2=blank_image, beta=1, gamma=0.0)
    return image

def process(image):
    height, width = image.shape[0], image.shape[1]
    region_of_interest_vertices = [(0, height), (width/2, height/2), (width, height)]
    gray_image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY)
    canny_image = cv2.Canny(image=gray_image, threshold1=250, threshold2=120)
    cropped_image = region_of_interest(image=canny_image, vertices=np.array([region_of_interest_vertices], np.int32))
    
    lines = cv2.HoughLinesP(image=cropped_image,
                            rho=2,
                            theta=np.pi / 180,
                            threshold=220,
                            lines=np.array([]),
                            minLineLength=150,
                            maxLineGap=5)
    drawing_lines = drawLines(image=image, lines=lines)
    return drawing_lines
while True:
    retval, img = video.read()
    if not retval: break

    try:
        img = process(image=img)
        cv2.imshow(winname="Video", mat=img)
    except: pass

    writer.write(image=img)
    if cv2.waitKey(20) == ord("q"): break
    
writer.release()
video.release()
cv2.destroyAllWindows()

In [14]:
?cv2.addWeighted

Docstring:
addWeighted(src1, alpha, src2, beta, gamma[, dst[, dtype]]) -> dst
.   @brief Calculates the weighted sum of two arrays.
.   
.   The function addWeighted calculates the weighted sum of two arrays as follows:
.   \f[\texttt{dst} (I)= \texttt{saturate} ( \texttt{src1} (I)* \texttt{alpha} +  \texttt{src2} (I)* \texttt{beta} +  \texttt{gamma} )\f]
.   where I is a multi-dimensional index of array elements. In case of multi-channel arrays, each
.   channel is processed independently.
.   The function can be replaced with a matrix expression:
.   @code{.cpp}
.       dst = src1*alpha + src2*beta + gamma;
.   @endcode
.   @note Saturation is not applied when the output array has the depth CV_32S. You may even get
.   result of an incorrect sign in the case of overflow.
.   @param src1 first input array.
.   @param alpha weight of the first array elements.
.   @param src2 second input array of the same size and channel number as src1.
.   @param beta weight of the second array eleme

In [30]:
image = np.zeros((500, 500, 3), dtype=np.uint8)

pts = np.array([[100, 100], [200, 400], [400, 200]], np.int32)

cv2.fillPoly(img=image, pts=[pts], color=(0, 0, 255))

cv2.imshow(winname="video", mat=image)
cv2.waitKey(0), cv2.destroyAllWindows()

(-1, None)